In [ ]:
https://jjxy.gdut.edu.cn/info/1071/6282.htm
https://jjxy.gdut.edu.cn/info/1071/6259.htm

https://jjxy.gdut.edu.cn/index/xw.htm
https://jjxy.gdut.edu.cn/index/xw/65.htm

<li class="clear wow fadeInUp"><a href="../info/1071/6282.htm" target="_blank" title="我院开展国家社科基金选题与写作规范专题讲座">我院开展国家社科基金选题与写作规范专题讲座<i>查看详情>></i></a><b>2023-11-02</b></li>


https://jjxy.gdut.edu.cn/__local/F/ED/5B/21369BDCE19D6A8FFC31BD73628_120A7E4D_51E4F.jpg

In [6]:
import requests
import bs4
import pandas as pd
import numpy as np
from docx import Document
import re
from docx.shared import Inches
from PIL import Image
from io import BytesIO
from docx.oxml.shared import OxmlElement, qn
from docx.enum.dml import MSO_THEME_COLOR
from docx.enum.text import WD_ALIGN_PARAGRAPH
import docx
from docx.enum.text import WD_UNDERLINE
from docx.shared import Pt
def add_hyperlink(paragraph, url, text, color='0000FF', underline=True):  #chat4生成的自定义函数，用于往word里添加超链接
    """
    A function that places a hyperlink within a paragraph object.

    :param paragraph: The Paragraph to which the hyperlink will be added.
    :param url: The URL of the hyperlink.
    :param text: The text to be displayed for the hyperlink.
    :param color: The color of the hyperlink - blue by default.
    :param underline: Whether the hyperlink is underlined - True by default.
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, 'http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink', is_external=True)

    # Create the w:hyperlink tag and set needed values
    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(qn('r:id'), r_id, )
    
    # Create a w:r element and a new w:rPr element
    new_run = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')

    # Add color if it is given
    if not color.startswith('00'):
        color = '00' + color
    c = OxmlElement('w:color')
    c.set(qn('w:val'), color)
    rPr.append(c)

    # Remove underlining if it is requested
    if not underline:
        u = OxmlElement('w:u')
        u.set(qn('w:val'), 'none')
        rPr.append(u)

    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

先去首页找/info/1071/6259.htm的内容
先尝试一个word文档的保存
总的顺序
找首页-》找单页-》提取单页的内容-》保存在word文档里输出

In [7]:

def getoneword(ulist,n):
    countzong=0
    regex_rule = r"/info/\d+/\d+\.htm"

    # 使用re.compile()编译为正则表达式对象
    pattern = re.compile(regex_rule)
    try:
        for i in ulist:
            countzong+=1
            try:
                i=re.search(pattern,i).group(0)
            except:
                continue
            url='https://jjxy.gdut.edu.cn'+i
            request=requests.get(url)
            request.encoding='UTF-8-SIG'
            soup=bs4.BeautifulSoup(request.text,'html.parser')
            a=soup.find('div',class_='show01').text.split('\n')
            b=soup.find('div',class_='show02')
            c=b.find_all('p')
            #获取图片的链接
            d=[]
            for i in b.find_all('img'):
                d.append(i['src'])

            #开始编写word
            doc=Document()
            # 设置全文的默认字体
            style = doc.styles['Normal']
            font = style.font
            font.name = 'Arial'
            font.size = Pt(12)
            style.element.rPr.rFonts.set(qn('w:eastAsia'), '宋体')#设置中文字体

            #插入超链接
            lianjie = doc.add_paragraph('按住ctrl')
            add_hyperlink(lianjie,url,'点击这里跳转')
            #插入标题
            P=doc.add_paragraph(f'{a[1]}')
            P.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 设置标题居中
            P_font = P.style.font
            P_font = P.runs[0].font  # 确保我们修改的是段落中第一个run的字体，有了这个就可以单独设置字体大小
            P_font.name = 'Arial'
            P_font.size = Pt(15)
            P.style.element.rPr.rFonts.set(qn('w:eastAsia'), '宋体')
            #插入时间
            T=doc.add_paragraph(f'{a[2]}')
            T.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 设置标题居中
            T_font = T.style.font
            T_font = T.runs[0].font
            T_font.name = 'Arial'
            T_font.size = Pt(10)
            T.style.element.rPr.rFonts.set(qn('w:eastAsia'), '宋体')
            #插入内容和图片,p1内容，p11图片下的文字
            count=0
            try:
                for i in c:

                    if i.find()==None:
                        p1=doc.add_paragraph(i.text)
                        p1.paragraph_format.first_line_indent = Inches(0.5)
                    else:
                        if i.find('img')==None:
                            p11=doc.add_paragraph(i.text)
                            p11.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 设置标题居中

                        else:

                            reimg=requests.get('https://jjxy.gdut.edu.cn/'+d[count])
                            img_data = BytesIO(reimg.content)
                            img = Image.open(img_data)
                            img.save(str(count)+'.png')
                            doc.add_picture(str(count)+'.png',width=Inches(6))
                            count+=1


                        print('\r当前总进度：{:.2f}%'.format(countzong*100/int(n*10)),end='')#打印进度
            except:
                doc.add_paragraph('格式出现错误')
            #使用 str.maketrans() 创建了一个字符映射表
            #使用 str.translate() 方法应用这个映射表
            safe_title = a[1].translate({ord(c): "-" for c in '\/:*?"<>|'})
    
            # 使用安全标题构造文件路径
            file_path = f'C:\\Users\\wen\\Desktop\\经院新闻合集\\{safe_title}.docx'
            doc.save(file_path)
    except:
        print(f'第{countzong+1}出错了')

In [8]:
def geturl(n,ulist=[]):
    #提取整页的所有链接url
    for i in range(n):
        if i==0:
            Url='https://jjxy.gdut.edu.cn/index/xw.htm'
            response=requests.get(Url)
            soup=bs4.BeautifulSoup(response.text,'html.parser')
            list1=soup.find_all('li',class_="clear wow fadeInUp")
            list2=[i.find()['href'].replace('.','').replace('htm','.htm') for i in list1]
            ulist=ulist+list2
        else:
            Url='https://jjxy.gdut.edu.cn/index/xw/'+str(66-i)+'.htm'
            response=requests.get(Url)
            soup=bs4.BeautifulSoup(response.text,'html.parser')
            list1=soup.find_all('li',class_="clear wow fadeInUp")
            list2=[i.find()['href'].replace('.','').replace('htm','.htm') for i in list1]
            ulist=ulist+list2
    return ulist
def main():

    n=int(input('爬取页数：'))
    getoneword(geturl(n,ulist=[]),n)

In [9]:
main()

爬取页数： 24


当前总进度：100.00%

In [87]:
geturl(3,ulist=[])

['/info/1071/6282.htm',
 '/info/1071/6259.htm',
 '/info/1071/6257.htm',
 '/info/1071/6225.htm',
 '/info/1071/6222.htm',
 '/info/1071/6216.htm',
 '/info/1071/6215.htm',
 '/info/1071/6192.htm',
 '/info/1071/6188.htm',
 '/info/1071/6185.htm',
 '//info/1071/6184.htm',
 '//info/1071/6183.htm',
 '//info/1071/6182.htm',
 '//info/1071/6174.htm',
 '//info/1071/6170.htm',
 '//info/1071/6169.htm',
 '//info/1071/6154.htm',
 '//info/1071/6153.htm',
 '//info/1071/6151.htm',
 '//info/1071/6150.htm',
 '//info/1071/6149.htm',
 '//info/1071/6147.htm',
 '//info/1071/6145.htm',
 '//info/1071/6196.htm',
 '//info/1071/6197.htm',
 '//info/1071/6125.htm',
 '//info/1071/6138.htm',
 '//info/1071/6136.htm',
 '//info/1071/6103.htm',
 '//info/1071/6104.htm']

In [101]:
kv={'user-agent':'Mozilla/5.0'}#标准浏览器的开头
url='https://jjxy.gdut.edu.cn'+'/info/1071/6282.htm'
request=requests.get('https://jjxy.gdut.edu.cn/info/1071/6225.htm',headers=kv)
request.encoding='UTF-8-SIG'
soup=bs4.BeautifulSoup(request.text,'html.parser')

In [102]:
a=soup.find('div',class_='show01').text.split('\n')
b=soup.find('div',class_='show02')
c=b.find_all('p')

In [107]:
c[3].find('img')


In [ ]:
url='https://jjxy.gdut.edu.cn/info/1071/6282htm'
https://jjxy.gdut.edu.cn/info/1071/6282.htm
<p style="text-align: center;"><span style="font-size: 14px;">冀祥德社长致辞</span></p>

In [94]:
regex_rule = r"/info/\d+/\d+\.htm"

# 使用re.compile()编译为正则表达式对象
pattern = re.compile(regex_rule)

In [97]:
re.search(pattern,'//info/1071/6145.htm').group()


'/info/1071/6145.htm'